Just experimenting right now

In [46]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
data_df = pd.read_json('./prepared_data/prepared_data.json', orient='records')
data_df.head()

,objectid,case_no,date,description,location,city,latitude,longitude,globalid,geohash,...,street_types,owners,max_width,min_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes,max_width_to_lane_ratio,min_width_to_lane_ratio
0,3,202200349491,2022-11-25 21:33:48+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.350226,-79.801527,f623ed61-fbc7-4391-8038-9e385d6ef3dd,hcyf42wd,...,"[MINOR, MAJOR]","[COB, ROH]",42.0,20.0,60,50,8,5,6.400000,3.333333
1,10,202200351471,2022-11-27 23:46:00+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.338309,-79.785323,aff64172-9cc4-44b0-a78b-71d0c21177c8,hcyf45tb,...,[MAJOR],[COB],35.0,24.0,60,50,6,5,7.000000,4.000000
2,13,202200353214,2022-11-29 15:40:00+00:00,MVC - HIT & RUN,GUELPH LI,BURLINGTON,43.364656,-79.822041,8883b2d6-9c65-4f61-a25d-5e9e3d77d4d9,hcycfuc4,...,[MAJOR],"[COB, ROH]",42.0,40.0,60,60,6,5,8.400000,7.000000
3,21,202200357807,2022-12-04 17:44:48+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.347329,-79.797437,48573cd9-b167-4c66-8bc7-7d6e32f546ee,hcyf43kd,...,[MAJOR],"[COB, ROH]",42.0,40.0,60,50,9,7,5.714286,4.444444
4,24,202200361004,2022-12-08 00:04:00+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.352178,-79.803863,6c97ac07-1a63-41b5-95f8-725dea1811f4,hcyf42pv,...,[MAJOR],"[MTO, ROH]",42.0,12.0,60,50,9,3,4.666667,4.000000


In [32]:
intersection_no_crash_df = pd.read_json('./prepared_data/intersection_info.json', orient='records')
print(intersection_no_crash_df.shape)
intersection_no_crash_df = intersection_no_crash_df[intersection_no_crash_df['geohash'].apply(lambda x: x not in data_df.geohash)]
intersection_no_crash_df.head()
intersection_no_crash_df['result'] = 0

(2517, 14)


Try and create a balance of street types

In [33]:
def get_highest(arr):
    if 'BOTH' in arr: 
        return 2#'BOTH'
    if 'MAJOR' in arr: 
        return 1#'MAJOR'
    if 'MINOR' in arr: 
        return 0#'MINOR'
data_df['highest_street_type'] = data_df.street_types.apply(get_highest)
data_df['result'] = 1
intersection_no_crash_df['highest_street_type'] = intersection_no_crash_df.street_types.apply(get_highest)

In [34]:
cols = ['max_width', 'min_width', 'max_speed_limit', 'min_speed_limit', 'max_lanes', 'min_lanes', 'highest_street_type', 'result']
model_data_df = pd.concat([data_df[cols], intersection_no_crash_df[cols].sample(n=data_df.shape[0])])

In [36]:
X = model_data_df.drop('result', axis=1)
y = model_data_df['result']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

Prepare the Model

In [58]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(len(cols)-1),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='MAE', metrics=['MSE'])
model.fit(x_train, y_train, epochs=30)
result = model.evaluate(x_test, y_test)
print(result)
print(math.sqrt(result[1]))

Epoch 1/30
47/47 [==============================] - 0s 804us/step - loss: 4.0671 - MSE: 37.7434
Epoch 2/30
47/47 [==============================] - 0s 891us/step - loss: 0.6192 - MSE: 0.6757
Epoch 3/30
47/47 [==============================] - 0s 1ms/step - loss: 0.3695 - MSE: 0.2474
Epoch 4/30
47/47 [==============================] - 0s 891us/step - loss: 0.2898 - MSE: 0.1748
Epoch 5/30
47/47 [==============================] - 0s 913us/step - loss: 0.2935 - MSE: 0.1744
Epoch 6/30
47/47 [==============================] - 0s 891us/step - loss: 0.2823 - MSE: 0.1694
Epoch 7/30
47/47 [==============================] - 0s 913us/step - loss: 0.2842 - MSE: 0.1661
Epoch 8/30
47/47 [==============================] - 0s 891us/step - loss: 0.3062 - MSE: 0.1796
Epoch 9/30
47/47 [==============================] - 0s 935us/step - loss: 0.2754 - MSE: 0.1620
Epoch 10/30
47/47 [==============================] - 0s 1ms/step - loss: 0.2738 - MSE: 0.1632
Epoch 11/30
47/47 [==============================] -